In [1]:
!pip list | grep gokart

gokart                        1.0.2


In [3]:
import gokart
import luigi

# Examples of using gokart at jupyter notebook

## Basic Usage
This is a very basic usage, just to dump a run result of ExampleTaskA.

In [4]:
class ExampleTaskA(gokart.TaskOnKart):
    param = luigi.Parameter()
    int_param = luigi.IntParameter(default=2)

    def run(self):
        self.dump(f'DONE {self.param}_{self.int_param}')

    
task_a = ExampleTaskA(param='example')
output = gokart.build(task=task_a)
print(output)

example_2


## Make tasks dependencies with `requires()`
ExampleTaskB is dependent on　ExampleTaskC and ExampleTaskD. They are defined in `ExampleTaskB.requires()`.

In [5]:
class ExampleTaskC(gokart.TaskOnKart):
    def run(self):
        self.dump('TASKC')
    
class ExampleTaskD(gokart.TaskOnKart):
    def run(self):
        self.dump('TASKD')

class ExampleTaskB(gokart.TaskOnKart):
    param = luigi.Parameter()

    def requires(self):
        return dict(task_c=ExampleTaskC(), task_d=ExampleTaskD())

    def run(self):
        task_c = self.load('task_c')
        task_d = self.load('task_d')
        self.dump(f'DONE {self.param}_{task_c}_{task_d}')
    
task_b = ExampleTaskB(param='example')
output = gokart.build(task=task_b)
print(output)

DONE example_TASKC_TASKD


## Make tasks dependencies with TaskInstanceParameter
The dependencies are same as previous example, however they are defined at the outside of the task instead of defied at `ExampleTaskB.requires()`.

In [6]:
class ExampleTaskC(gokart.TaskOnKart):
    def run(self):
        self.dump('TASKC')
    
class ExampleTaskD(gokart.TaskOnKart):
    def run(self):
        self.dump('TASKD')

class ExampleTaskB(gokart.TaskOnKart):
    param = luigi.Parameter()
    task_1 = gokart.TaskInstanceParameter()
    task_2 = gokart.TaskInstanceParameter()

    def requires(self):
        return dict(task_1=self.task_1, task_2=self.task_2)  # required tasks are decided from the task parameters `task_1` and `task_2`

    def run(self):
        task_1 = self.load('task_1')
        task_2 = self.load('task_2')
        self.dump(f'DONE {self.param}_{task_1}_{task_2}')
    
task_b = ExampleTaskB(param='example', task_1=ExampleTaskC(), task_2=ExampleTaskD())  # Dependent tasks are defined here
output = gokart.build(task=task_b)
print(output)

DONE example_TASKC_TASKD
